In [113]:
import pandas as pd
import numpy as np

# Funtion and Constants Definition

In [21]:
COLS = ["type", "time (ms)"]

In [114]:
def remove_outlier_using_turkey_method(ser: pd.Series):
    q1 = ser.quantile(0.25)
    q3 = ser.quantile(0.75)
    iqr = q3 - q1
    print(q1, q3, iqr)
    lb = q1 - 1.5 * iqr
    ub = q3 + 1.5 * iqr
    return ser[(ser >= lb) & (ser <= ub)]

def get_real_rtt(df_raw_rtt: pd.DataFrame, df_process_latency: pd.DataFrame):
    time = COLS[1]
    ser = pd.Series()
    ser = df_raw_rtt[time] - df_process_latency[time]
    return ser

# ZeroMQ RTT and Latency

This section calculates the RTT (round trip time) and latency of ZeroMQ. Latency is approximated with the formula:
$$
\text{latency} \approx \frac{\text{RTT}}{2}
$$

The round trip time is
$$
\text{RTT} = \text{send a sensor data time} - \text{process time} + \text{send control time} 
$$
For testing, the sensor that will be used is camera. This is because only 2 sensors are used: GNSS and camera and camera has the largest data size between the two.

In [100]:
raw_rtt_1 = pd.read_csv("./raw_rtt_001.csv", names=COLS, header=None)
raw_rtt_2 = pd.read_csv("./raw_rtt_002.csv", names=COLS, header=None)
process_latency_1 = pd.read_csv("./process_latency_001.csv", names=COLS, header=None)
process_latency_2 = pd.read_csv("./process_latency_002.csv", names=COLS, header=None)

In [101]:
raw_rtt_1.tail()

,type,time (ms)
11775,ZEROMQ_RAW_RTT_11775,57
11776,ZEROMQ_RAW_RTT_11776,56
11777,ZEROMQ_RAW_RTT_11777,55
11778,ZEROMQ_RAW_RTT_11778,59
11779,ZEROMQ_RAW_RTT_11779,58


In [118]:
real_rtt_1 = get_real_rtt(raw_rtt_1, process_latency_1)
real_rtt_1 = remove_outlier_using_turkey_method(real_rtt_1)

real_rtt_2 = get_real_rtt(raw_rtt_2, process_latency_2)
real_rtt_2 = remove_outlier_using_turkey_method(real_rtt_2)

real_rtt = np.append(real_rtt_1.to_numpy(), real_rtt_2.to_numpy())
real_rtt = pd.Series(real_rtt)
real_rtt.describe()

9.0 10.0 1.0
9.0 10.0 1.0


count    15858.000000
mean         9.597742
std          0.685207
min          8.000000
25%          9.000000
50%         10.000000
75%         10.000000
max         11.000000
dtype: float64

In [117]:
latencies = real_rtt / 2
latencies.describe()

count    15858.000000
mean         4.798871
std          0.342604
min          4.000000
25%          4.500000
50%          5.000000
75%          5.000000
max          5.500000
dtype: float64

# Old HILS "Scientific" Latency

This section calculates the latency of the old HILS connector implementation. The details of the implementation can be seen in the paper or the book. This part only calculates latency during the process of sending data from a "sender" (produces sensor data) to a "receiver" (consumer of sensor data). The latency is only scientific and not real because the HTTP latency is not accounted for.